In [131]:
#어플에서 input되는 값
birthday = '9810062'
diseaseCode = 'I499'
emergency = False

# create-model

In [132]:
#NANONETS를 사용하기 위해 필요한 API KEY와 MODEL 할당
NANONETS_API_KEY = 'YzUkAjjFJsQndp-09EOXR0woutkN8FjG'
NANONETS_MODEL_ID = '37baa2d5-c5a2-4ada-9d3f-07eb99288cca'

model_id = NANONETS_MODEL_ID
api_key = NANONETS_API_KEY

In [133]:
import requests, os, json
  
url = "https://app.nanonets.com/api/v2/ObjectDetection/Model/"
payload = "{\"categories\" : [\"number_plate\"], \"model_type\": \"ocr\"}"
headers = {'Content-Type': "application/json",}

response = requests.request("POST", url, headers=headers, auth=requests.auth.HTTPBasicAuth(api_key, ''), data=payload)

# upload-training

In [134]:
import os, requests, json
from tqdm import tqdm

pathToAnnotations = './annotations/json'
pathToImages = './images'

for root, dirs, files in os.walk(pathToAnnotations, topdown=False):
    for name in tqdm(files):
        annotation = open(os.path.join(root, name), "r")
        filePath = os.path.join(root, name)
        imageName, ext = name.split(".")
        if imageName == "":
            continue
        imagePath = os.path.join(pathToImages, imageName + '.jpg')
        jsonData = annotation.read()
        url = 'https://app.nanonets.com/api/v2/ObjectDetection/Model/' + model_id + '/UploadFile/'
        data = {'file' :open(imagePath, 'rb'),  'data' :('', '[{"filename":"' + imageName+".jpg" + '", "object": '+ jsonData+'}]'),   'modelId' :('', model_id)}       
        response = requests.post(url, auth=requests.auth.HTTPBasicAuth(api_key, ''), files=data)
        if response.status_code > 250 or response.status_code<200:
            print(response.text), response.status_code

# train-model

In [135]:
import requests, os

url = 'https://app.nanonets.com/api/v2/ObjectDetection/Model/' + model_id + '/Train/'
querystring = {'modelId': model_id}
response = requests.request('POST', url, auth=requests.auth.HTTPBasicAuth(api_key, ''), params=querystring)

# Model State

In [136]:
import requests, os, json
model_id = NANONETS_MODEL_ID
api_key = NANONETS_API_KEY

url = 'https://app.nanonets.com/api/v2/ObjectDetection/Model/' + model_id
response = requests.request('GET', url, auth=requests.auth.HTTPBasicAuth(api_key,''))
state = json.loads(response.text)["state"]
status = json.loads(response.text)["status"]

if state != 5:
    print("The model isn't ready yet, its status is:", status)
    print("We will send you an email when the model is ready. If you are impatient, run this script again in 10 minutes to check.")
else:
    print("NEXT RUN: python ./code/prediction.py ./images/151.jpg")

NEXT RUN: python ./code/prediction.py ./images/151.jpg


# Model Predict

In [137]:
image_path = r"docImage.jpg"

In [138]:
import requests, os, sys
import json
import unicodedata
import time

# model_id = NANONETS_MODEL_ID
# api_key = NANONETS_API_KEY

#서버에서 결과를 보내주기 때문에 파일 형식은 JSON
url = 'https://app.nanonets.com/api/v2/ObjectDetection/Model/' + model_id + '/LabelFile/'

data = {'file': open(image_path, 'rb'),'modelId': ('', model_id)}

response = requests.post(url, auth=requests.auth.HTTPBasicAuth(api_key, ''), files=data)

#Json response parsing
with open("data.json", "w") as f:
    json.dump(response.json(), f)
data = json.load(open("data.json"))

In [139]:
diagnosisList = []
patientList = []

#0번째 테이블 내에 있는 data 
for i in data['result'][0]['prediction'][0]['cells']:
    patientList.append(i['text'])
patientList.reverse()

#1번째 테이블 내에 있는 data 
for i in data['result'][0]['prediction'][1]['cells']:
    diagnosisList.append(i['text'])
diagnosisList.reverse()

#가져온 리스트를 txt파일에 저장, 문제가 있을 경우 확인하기 위함임.
with open("patient.txt", "w") as patient:
    for item in patientList:
        patient.write("%s\n" % item)
        
with open('diagnosis.txt', 'w') as diagnosis:
    for item in diagnosisList:
        diagnosis.write("%s\n" % item)

In [140]:
#데이터프레임으로 처리하기
import pandas as pd

diagnosisDF = pd.DataFrame()
patientDF = pd.DataFrame()

In [141]:
#patientList DataFrame으로 만들기.
for t in range((len(patientList))//7): #patient의 column의 수는 7개
    patientAppendList = []
    for i, item in enumerate(patientList):
         if t*7<= i < (t+1)*7:
                patientAppendList.append(item)
    appendSeries = pd. Series(patientAppendList)
    patientDF = patientDF. append(appendSeries, ignore_index=True)
    
#보건복지부 양식이기에 column명은 동일
patientDF.columns = ["환자 등록 번호","환자 성명","진료 기간","병실","진료과목","보험자구분","구분"]
patientDF = patientDF.iloc[1:]
patientDF

,환자 등록 번호,환자 성명,진료 기간,병실,진료과목,보험자구분,구분
1,10041004,이규영,2020.11.28,601,내과,건강 보험,


In [142]:
#diagnosisList DataFrame으로 만들기.
for t in range((len(diagnosisList))//11): #diagnosis의 column의 수는 11개
    diagnosisAppendList = []
    for i, item in enumerate(diagnosisList):
        if t*11<= i < (t+1)*11:
            diagnosisAppendList.append(item)
    print(diagnosisAppendList)
    appendSeries = pd. Series(diagnosisAppendList)
    diagnosisDF = diagnosisDF. append(appendSeries, ignore_index=True)
    
    
#보건복지부 양식이기에 column은 동일
diagnosisDF.columns = ['일자', '분류코드(기관)', '명칭', '금액', '횟수', '일수', '총액', '공단부담', '본인부담', '전액본인부담', '비급여']
diagnosisDF

['일자', '코드', '명칭', '금액', '횟수', '일수', '총액', '일부\n공단 부담', '급여\n본인 부담\n본인 부담', '전액 본인', '비급여']
['0104 투약', '및 조제료', '', '', '', '', '', '', '', '', '']
['2020-11-28', '645402940', '모사 메드 정 5 밀리그램', '95', '3.0000', '1', '285', '', '', '', '285']
['2020-11-28', '645403580', '메트 마 릴정 1/250 밀리그램', '132', '2.0000', 'Z', '528', '', '', '', '528']
['2020-11-28', '659900340', '아큐 판 주사액 20mg ( 염산 네 포팜 )', '103', '3.0000', '1', '309', '', '', '', '309']
['2020-11-28', '621802460', '비스타 치 정 50mg ( 파비스 )', '56', '3', 'Z', '336', '', '', '', '336']


,일자,분류코드(기관),명칭,금액,횟수,일수,총액,공단부담,본인부담,전액본인부담,비급여
0,일자,코드,명칭,금액,횟수,일수,총액,일부\n공단 부담,급여\n본인 부담\n본인 부담,전액 본인,비급여
1,0104 투약,및 조제료,,,,,,,,,
2,2020-11-28,645402940,모사 메드 정 5 밀리그램,95,3.0000,1,285,,,,285
3,2020-11-28,645403580,메트 마 릴정 1/250 밀리그램,132,2.0000,Z,528,,,,528
4,2020-11-28,659900340,아큐 판 주사액 20mg ( 염산 네 포팜 ),103,3.0000,1,309,,,,309
5,2020-11-28,621802460,비스타 치 정 50mg ( 파비스 ),56,3,Z,336,,,,336


In [143]:
#중분류숫자만으로 중분류코드 완성하기
Dic = {'0308': '0308 검사료',
 '0311': '0311 치료재료대',
 '0318': '0318 초음파진단료',
 '0320': '0320 기타',
 '0305': '0305 주사료',
 '0108': '0108 검사료',
 '0316': '0316 MRI 진단료',
 '0307': '0307 처치 및 수술료',
 '0304': '0304 투약 및 조제료',
 '0312': '0312 재활 및 물리치료료',
 '0105': '0105 주사료',
 '0301': '0301 진찰료',
 '0208': '0208 검사료',
 '0309': '0309 영상진단료',
 '0201': '0201 진찰료',
 '0104': '0104 투약 및 조제료',
 '0101': '0101 진찰료',
 '0321': '0321 의료기관미징수료(원외약제)',
 '0112': '0112 재활 및 물리치료료',
 '0209': '0209 영상진단료',
 '0121': '0121 의료기관미징수료(원외약제)',
 '0120': '0120 기타',
 '0109': '0109 영상진단료',
 '0315': '0315 CT 진단료',
 '0111': '0111 치료재료대',
 '0215': '0215 CT 진단료',
 '0302': '0302 입원료',
 '0107': '0107 처치 및 수술료',
 '0306': '0306 마취료',
 '0116': '0116 MRI 진단료',
 '0118': '0118 초음파진단료',
 '0102': '0102 입원료',
 '0310': '0310 방사선치료료',
 '0207': '0207 처치 및 수술료',
 '0216': '0216 MRI 진단료',
 '0210': '0210 방사선치료료',
 '0218': '0218 초음파진단료',
 '0106': '0106 마취료',
 '0115': '0115 CT 진단료',
 '0220': '0220 기타',
 '0217': '0217 PET 진단료',
 '0303': '0303 식대',
 '0117': '0117 PET 진단료',
 '0213': '0213 정신요법료',
 '0317': '0317 PET 진단료',
 '0202': '0202 입원료',
 '0206': '0206 마취료',
 '0319': '0319 보철ㆍ교정료',
 '0313': '0313 정신요법료',
 '0110': '0110 방사선치료료',
 '0113': '0113 정신요법료',
 '0114': '0114 전혈 및 혈액성분제제료',
 '0205': '0205 주사료',
 '0314': '0314 전혈 및 혈액성분제제료',
 '0211': '0211 치료재료대',
 '0212': '0212 재활 및 물리치료료',
 '0103': '0103 식대',
 '0119': '0119 보철ㆍ교정료'}

In [144]:
#중간에 중분류가 들어가 있기 때문에 이를 탐지해야함.
#날짜 형식에 맞지 않으면 중분류임

midSeper = ""
midIndex = 0
for index, item in enumerate(diagnosisDF['일자']):
    if len(item.split('-')) != 3 :
        midSeper = item
        midIndex = index
        
#테이블로 읽다보니 따로따로 나뉘어지기 때문
midSeper = ""
for i in diagnosisDF.iloc[midIndex]:
    midSeper += i

#코드만으로 중분류 완성하기
SeperCode = ""
for i in midSeper :
    if i.isdigit():
        SeperCode += i
midSeper = Dic[SeperCode]

In [145]:
diagnosisDF = diagnosisDF.iloc[midIndex+1:] #중분류 뒷 부분만 가져오기. = 내용부분임

In [146]:
diagnosisDF.drop(['금액', '횟수','일수','총액','공단부담','본인부담','전액본인부담'], axis = 1, inplace = True)

/Users/jubykim/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [147]:
#환자정보 중 predict에 사용되는 항목 가져오기
diagnosisDF['진료과목'] = patientDF['진료과목'][1]
diagnosisDF['보험자구분'] = patientDF['보험자구분'][1]

<ipython-input-147-3f24b7c4605d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnosisDF['진료과목'] = patientDF['진료과목'][1]
<ipython-input-147-3f24b7c4605d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnosisDF['보험자구분'] = patientDF['보험자구분'][1]


In [148]:
diagnosisDF['항코드(중분류)'] = midSeper

<ipython-input-148-b41e27ed970a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnosisDF['항코드(중분류)'] = midSeper


In [149]:
#진료비 세부내역서를 읽은 최종 DF
diagnosisDF

,일자,분류코드(기관),명칭,비급여,진료과목,보험자구분,항코드(중분류)
2,2020-11-28,645402940,모사 메드 정 5 밀리그램,285,내과,건강 보험,0104 투약 및 조제료
3,2020-11-28,645403580,메트 마 릴정 1/250 밀리그램,528,내과,건강 보험,0104 투약 및 조제료
4,2020-11-28,659900340,아큐 판 주사액 20mg ( 염산 네 포팜 ),309,내과,건강 보험,0104 투약 및 조제료
5,2020-11-28,621802460,비스타 치 정 50mg ( 파비스 ),336,내과,건강 보험,0104 투약 및 조제료


In [150]:
#그 중에서도 비급여 내역만을 추출 = 비급여 지출내역이 있는 항목만.
non_benefit_DF = diagnosisDF[diagnosisDF['비급여'] != ''] #비급여내역만을 추출

In [151]:
non_benefit_DF.drop('비급여', axis = 1, inplace = True)
non_benefit_DF.reset_index(inplace = True)
non_benefit_DF.drop('index', axis = 1, inplace = True)

In [152]:
#학습을 위한 column 생성

#1. 성별
if int(birthday[:-1]) % 2 == 0:
    sex = 0
else : sex = 1
    
non_benefit_DF['환자성별'] = sex

In [153]:
#주민번호 뒷자리 첫 번째 숫자를 통해 몇 세기에 태어났는지 판별
if birthday[-1] == '2' or '1':
    fullBirthday = '19' + birthday
elif birthday[-1] == '3' or '4':
    fullBirthday = '20' + birthday
elif birthday[-1] == '5' or '6':
    fullBirthday = '19' + birthday
elif birthday[-1] == '7' or '8':
    fullBirthday = '20' + birthday
elif birthday[-1] == '9' or '':
    fullBirthday = '18' + birthday

non_benefit_DF['환자생년월일'] = fullBirthday

birthY = fullBirthday[:4]
birthM = fullBirthday[4:6]
birthD = fullBirthday[7:]

In [154]:
# 환자가 진료를 받았을 당시의 만 나이 구하기
diaY, diaM, diaD = non_benefit_DF['일자'][0].split('-')

diaY, diaM, diaD = int(diaY),int(diaM),int(diaD)
birthY, birthM, birthD = int(birthY),int(birthM),int(birthD)

if diaM > birthM :
    year = diaY - birthY
elif diaM < birthM :
    year = diaY-birthY -1
else :
    if diaD >= birthD :
        year = diaY - birthY
    else : 
        year = diaY - birthY -1
        
non_benefit_DF['환자 만 나이'] = year
non_benefit_DF.drop('환자생년월일', axis = 1, inplace = True)
non_benefit_DF.drop('일자', axis = 1, inplace = True)

In [155]:
#연도 + 반기 column만들기
half = 0
if diaM < 7 :
    half = 1
else :
    half = 2
yearHalf = str(diaY) + str(half)

non_benefit_DF['연도 + 반기'] = yearHalf

In [156]:
non_benefit_DF['상병기호'] = diseaseCode

In [157]:
# diagnosisDF['구분'] 

if emergency :
    non_benefit_DF['구분'] = '긴급의료지원비확인'
else :
    non_benefit_DF['구분'] = '요양급여대상여부 확인'

In [158]:
non_benefit_DF

,분류코드(기관),명칭,진료과목,보험자구분,항코드(중분류),환자성별,환자 만 나이,연도 + 반기,상병기호,구분
0,645402940,모사 메드 정 5 밀리그램,내과,건강 보험,0104 투약 및 조제료,0,22,20202,I499,요양급여대상여부 확인
1,645403580,메트 마 릴정 1/250 밀리그램,내과,건강 보험,0104 투약 및 조제료,0,22,20202,I499,요양급여대상여부 확인
2,659900340,아큐 판 주사액 20mg ( 염산 네 포팜 ),내과,건강 보험,0104 투약 및 조제료,0,22,20202,I499,요양급여대상여부 확인
3,621802460,비스타 치 정 50mg ( 파비스 ),내과,건강 보험,0104 투약 및 조제료,0,22,20202,I499,요양급여대상여부 확인


# ISSUE1 : 의원의 정보는 NanoNets에서 읽어오지 못함

### 아이디어 : 
환자정보가 들어있는 table이 제일 처음 나오는 것과, 환자등록번호 column의 크기는 늘 동일하므로, 해당 셀을 기준으로 의원정보가 담겨있는 셀을 가져오자

In [159]:
pip install openCV-python

Note: you may need to restart the kernel to use updated packages.


In [160]:
#잘 읽어올 수 있도록 openCV를 통해 ersion,morphologyEx, dilate 등의 과정을 거친다

import cv2
import numpy as np
def detect_box(image,line_min_width=1):
    gray_scale=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    th1,img_bin = cv2.threshold(gray_scale,150,225,cv2.THRESH_BINARY)
    kernal6h = np.ones((1,line_min_width), np.uint8)
    kernal6v = np.ones((line_min_width,1), np.uint8)
    img_bin_h = cv2.morphologyEx(~img_bin, cv2.MORPH_OPEN, kernal6h)
    img_bin_v = cv2.morphologyEx(~img_bin, cv2.MORPH_OPEN, kernal6v)
    img_bin_final=img_bin_h|img_bin_v
    final_kernel = np.ones((3,3), np.uint8)
    img_bin_final=cv2.dilate(img_bin_final,final_kernel,iterations=1)
    ret, labels, stats,centroids = cv2.connectedComponentsWithStats(~img_bin_final, connectivity=8, ltype=cv2.CV_32S)
    return stats,labels

In [161]:
from PIL import Image

In [162]:
import cv2

image_path = "docImage.jpg"
image=cv2.imread(image_path)
stats,labels=detect_box(image)

xList = []
yList = []
wList = []
hList = []
areaList = []

for x,y,w,h,area in stats:
    xList.append(x)
    yList.append(y)
    wList.append(w)
    hList.append(h)
    areaList.append(area)

In [163]:
#환자 등록번호 cell의 칼럼 area는 고정이므로 해당 셀을 기준으로 요양기호셀을 찾고자 함.
#환자관련 table은 늘 첫 테이블의 첫번째 셀이므로 알고리즘에 따라 index가 2일 때 환자등록번호라는 셀이 나옴.
# 0번째는 사진전체, 1번째는 세부내역서 내용이 있는 부분만, 2번째는 첫 번째 셀
x = xList[12]
y = yList[12]
h = hList[12]
w = wList[12]
pkArea = areaList[12]
crop_img = image[y:y+h, x:x+w]

#환자등록번호 cell의 칼럼.
cv2.imwrite('patientPK.jpg',crop_img)
pk=cv2.imread('patientPK.jpg')

In [164]:
# testImage 기준, 요양기호정보cell대비 환자등록번호 cell의 크기는 0.42었음.
# 사용자가 사진을 찍는 방향이나 각도에 따라 달라질 수 있으니 적당한 범위를 정해준다.
for index , item in enumerate(areaList):
    if 0.3 < pkArea / item < 0.55:
        x = xList[index]
        y = yList[index]
        h = hList[index]
        w = wList[index]

crop_img = image[y:y+h, x:x+w]

#요양정보 cell의 칼럼.
cv2.imwrite('hospCode.jpg',crop_img)
hospCode=cv2.imread('hospCode.jpg')

# ISSUE2 : 글씨에 비해 배경이 너무 많을 경우 잘 인식하지 못하므로 글씨있는 부분만 crop

In [165]:
from PIL import Image, ImageChops
 
def crop(im):
    background = Image.new(im.mode, im.size, im.getpixel((0, 0)))
    diff = ImageChops.difference(im, background)
    diff = ImageChops.add(diff, diff, 2.0, -35)
    bbox = diff.getbbox()
    if bbox:
        cropped_image = im.crop(bbox)
        return cropped_image
    else:
        print('Failure!')
        return

In [166]:
an_image = Image.open('hospCode.jpg')
image_sequence = an_image.getdata()
image_array = np.array(image_sequence)

In [167]:
#글자가 있는 부분만 crop
croppedImage = crop(an_image)

In [168]:
#invalid argument는 PIL상의 bug이고, 다시 한 번 실행 시 제대로 동작합니다. 다시 한번 실행해주세요.
croppedImage.save('croppedImage.jpg')

In [169]:
pip install google-cloud-vision

Note: you may need to restart the kernel to use updated packages.


In [170]:
#이미지 내에서 글씨를 detection하기.
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    import os
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"meari-322801-754104bf5e4a.json"
    client = vision.ImageAnnotatorClient()
    textList = []
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations

    for text in texts:
        textList.append('\n"{}"'.format(text.description))
        
    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    return textList

In [171]:
hospNum = ""
for i in detect_text('croppedImage.jpg')[0]:
    if i.isdigit():
        hospNum += i
hospNum

'21100390'

In [172]:
non_benefit_DF['요양기호'] = hospNum

In [173]:
non_benefit_DF

,분류코드(기관),명칭,진료과목,보험자구분,항코드(중분류),환자성별,환자 만 나이,연도 + 반기,상병기호,구분,요양기호
0,645402940,모사 메드 정 5 밀리그램,내과,건강 보험,0104 투약 및 조제료,0,22,20202,I499,요양급여대상여부 확인,21100390
1,645403580,메트 마 릴정 1/250 밀리그램,내과,건강 보험,0104 투약 및 조제료,0,22,20202,I499,요양급여대상여부 확인,21100390
2,659900340,아큐 판 주사액 20mg ( 염산 네 포팜 ),내과,건강 보험,0104 투약 및 조제료,0,22,20202,I499,요양급여대상여부 확인,21100390
3,621802460,비스타 치 정 50mg ( 파비스 ),내과,건강 보험,0104 투약 및 조제료,0,22,20202,I499,요양급여대상여부 확인,21100390


In [174]:
forDic = [] #사전에 돌려볼 명칭 리스트
for i in non_benefit_DF['명칭']:
    forDic.append(i.replace(" ", ""))
non_benefit_DF = non_benefit_DF.drop('명칭', axis = 1)

In [175]:
forMLDF = []
for row in range(len(non_benefit_DF.index)) :
    forMLDF.append(non_benefit_DF.iloc[row])

In [176]:
forMLDF #머신러닝할 때 쓸 DF List, 한줄씩출력임.

[분류코드(기관)        645402940
 진료과목                   내과
 보험자구분               건강 보험
 항코드(중분류)    0104 투약 및 조제료
 환자성별                    0
 환자 만 나이                22
 연도 + 반기             20202
 상병기호                 I499
 구분            요양급여대상여부 확인
 요양기호             21100390
 Name: 0, dtype: object,
 분류코드(기관)        645403580
 진료과목                   내과
 보험자구분               건강 보험
 항코드(중분류)    0104 투약 및 조제료
 환자성별                    0
 환자 만 나이                22
 연도 + 반기             20202
 상병기호                 I499
 구분            요양급여대상여부 확인
 요양기호             21100390
 Name: 1, dtype: object,
 분류코드(기관)        659900340
 진료과목                   내과
 보험자구분               건강 보험
 항코드(중분류)    0104 투약 및 조제료
 환자성별                    0
 환자 만 나이                22
 연도 + 반기             20202
 상병기호                 I499
 구분            요양급여대상여부 확인
 요양기호             21100390
 Name: 2, dtype: object,
 분류코드(기관)        621802460
 진료과목                   내과
 보험자구분               건강 보험
 항코드(중분류)    0104 투약 및 조제료
 환자성별  

In [177]:
forDic

['모사메드정5밀리그램', '메트마릴정1/250밀리그램', '아큐판주사액20mg(염산네포팜)', '비스타치정50mg(파비스)']

# NLP

In [178]:
pip install JPype1

Note: you may need to restart the kernel to use updated packages.


In [179]:
pip install rhinoMorph

Note: you may need to restart the kernel to use updated packages.


In [180]:
import pandas as pd
import numpy as np
import re
import rhinoMorph

# Text proprecessing
- input 명칭에 대한 텍스트전처리

In [181]:
def input_proprecessing(data): # 텍스트전처리에 대한 함수
    dic = []
    for i in data:  # 특수기호 제거
        text = re.sub('[-_=+,/\?:^.*·''""(\)[\]<\>「\」|`~!%βⅡ°\ © ® ° α β γ δ ε ζ η θ ι κ λ μ π ο φ χ ψ ω Ф Э ф • ⁿ ‥ … € Ⅰ Ⅱ Ⅲ Ⅳ Ⅴ Ⅵ ⅦⅧ ™ Ω ⅓ ⅔ ℃ ℓ → ← ∙ ① ② ③ ④ ⑤ ⑥ ⑦ ⑧ ⑨ ⑩ ■ □ ▲ △ ▶ ◆◇ ◈ ◎ ○ ● ◐ ◑ ★ ☆ ♠ ♡ ♣ ♤ ♥ 〮 ㎀ ㎁ ㎂ ㎃ ㎄ ㎈ ㎉ ㎊ ㎋ ㎌ ㎍㎎ ㎕ ㎖ ㎗ ㎘ ㎚ ㎛ ㎜ ㎝ ㎞ ㎟ ㎠ ㎡ ㎢ ㎣ ㎤ ㎥ ㎦ ㎧ ㎼ ㎽ ㎾ ￠ ￡ ～ ︒ ︓ ⅓™αμΦ×,Ⅲ∥Ⅰ~ⅣΙ®ⅤⅥⅦ㎛*Øⓜ＊ㆍ?◈°ⅠⅡⅢⅣΙΙβ]', ' ', i)
        dic.append(text) # 특수기호 제거한 것들을 dic 리스트에 추가
    rn = rhinoMorph.startRhino()  # 형태소분석기 객체 생성
    dic_list = []
    for data_each in dic:  # 전체 데이터 Tokenizer
        morphed_data_each = rhinoMorph.onlyMorph_list(rn, data_each, pos =['NNG', 'NNP','SL'], eomi = True)  # 명칭이므로 일반명사, 고유명사, 영어태그만 사용
        dic_list.append(morphed_data_each)  # 형태소 분석한 값들을 추가
    f_list = []
    for i in dic_list:    # 중복값 제거 -> 차후 점수합산에 방해요인
        set_list = set(i)
        final_list = list(set_list)  
        f_list.append(final_list)
    return f_list

In [182]:
result = input_proprecessing(forDic)
result

filepath:  /Users/jubykim/opt/anaconda3/lib/python3.8/site-packages
classpath:  /Users/jubykim/opt/anaconda3/lib/python3.8/site-packages/rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!


[['메드', '모사', '밀리그램', '정'], [], ['염산', 'mg'], ['mg']]

# USE DICTIONARY
- 사전을 불러와 급여 비급여에 대한 점수를 가산

In [183]:
def read_data(filename, encoding = 'UTF-8'):   # 사전 읽기 함수
    with open(filename, 'r', encoding = encoding) as f:  
        data = [line.split('\t') for line in f.read().splitlines()] 
        data = data[0]       
    return data   

In [184]:
insurance = read_data("insurance.txt", encoding='UTF-8')  # 급여 사전 불러오기
insurance_non = read_data("insurance_non.txt", encoding='UTF-8')  # 비급여 사전 불러오기

In [185]:
def Use_Dictionary(data, dictionary):   # 급여 비급여사전 이용하기
    dictionary_found = []
    for onedata in data:
        oneline_word = onedata.split(' ')  # 한 줄의 데이터를 공백 단위로 분리하여 리스트로 저장
        dictionary_temp = 0    # 일치하는 횟수 count에 사용               
        for dict_word in dictionary:            # 사전의 어휘
            if dict_word in oneline_word:
                dictionary_temp += 1                # 현재의 단어와 일치하면 1점씩 가산 (중복X)
        dictionary_found.append(dictionary_temp)     # 현재의 줄에서 찾은 감성단어의 숫자를 해당 위치에 저장
    dictionary_sum = sum(dictionary_found)       # 총합
    return dictionary_sum

In [186]:
ins_list = [] # 급여 항목에서의 총합
for i in result:
    ins = Use_Dictionary(i, insurance)
    ins_list.append(ins)
ins_list = np.array(ins_list)  # 점수를 보기위해 array형태로 변환

In [187]:
non_ins_list = []  # 비급여 항목에서의 총합
for i in result: 
    non_ins = Use_Dictionary(i, insurance_non)
    non_ins_list.append(ins)
non_ins_list = np.array(non_ins_list) 

In [188]:
print('급여점수: ' ,ins_list)
print('비급여점수: ' ,non_ins_list)

급여점수:  [2 0 2 1]
비급여점수:  [1 1 1 1]


In [189]:
score_list = ins_list - non_ins_list # 총 점수
num = 0   # 명칭개수에 대한 count
for score in score_list:
    num += 1
    if score > 0:
        print(f'{num}번째 결과: 1')  # 양수이면 급여항목이므로 환불대상
    else:
        print(f'{num}번째 결과: 0') # 그 이외의 경우는 ML모델로 넘어감

1번째 결과: 1
2번째 결과: 0
3번째 결과: 1
4번째 결과: 0


In [190]:
score_list  # 총 점수

array([ 1, -1,  1,  0])

# ML model

In [191]:
entryForML = []
positiveScore = []
score_list = score_list.tolist()

In [192]:
#사전결과 0보다 작거나 같은 항목들만 ML에서 predict
for i in range(0,len(score_list)):
    if score_list[i] <= 0:
        entryForML.append(i)
    else:
        positiveScore.append(i)

In [193]:
positiveScore

[0, 2]

In [194]:
# 사전에서 걸러진 급여항목 출력
for paid_idx in positiveScore:
    print(str(paid_idx) + '번째 항목은 급여일 가능성이 매우 높습니다. 신청을 원하시면 제출버튼을 눌러주세요.')

0번째 항목은 급여일 가능성이 매우 높습니다. 신청을 원하시면 제출버튼을 눌러주세요.
2번째 항목은 급여일 가능성이 매우 높습니다. 신청을 원하시면 제출버튼을 눌러주세요.


In [195]:
entryForML

[1, 3]

In [196]:
dfForML = non_benefit_DF.iloc[entryForML]

In [197]:
dfForML['보험자구분'] = dfForML['보험자구분'].apply(lambda x:x.replace(' ', ''))

<ipython-input-197-f0faf9ab7951>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfForML['보험자구분'] = dfForML['보험자구분'].apply(lambda x:x.replace(' ', ''))


In [198]:
dfForML

,분류코드(기관),진료과목,보험자구분,항코드(중분류),환자성별,환자 만 나이,연도 + 반기,상병기호,구분,요양기호
1,645403580,내과,건강보험,0104 투약 및 조제료,0,22,20202,I499,요양급여대상여부 확인,21100390
3,621802460,내과,건강보험,0104 투약 및 조제료,0,22,20202,I499,요양급여대상여부 확인,21100390


In [199]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [200]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [201]:
pip install warnings

ERROR: Could not find a version that satisfies the requirement warnings
ERROR: No matching distribution found for warnings
Note: you may need to restart the kernel to use updated packages.


In [202]:
pip install pickle

ERROR: Could not find a version that satisfies the requirement pickle
ERROR: No matching distribution found for pickle
Note: you may need to restart the kernel to use updated packages.


In [203]:
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, roc_auc_score, f1_score
from sklearn.metrics import classification_report
import warnings
import pickle

In [204]:
np_load_old = np.load
np.load = lambda *a,**k : np_load_old(*a, allow_pickle=True)

dfForML.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 1 to 3
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   분류코드(기관)  2 non-null      object
 1   진료과목      2 non-null      object
 2   보험자구분     2 non-null      object
 3   항코드(중분류)  2 non-null      object
 4   환자성별      2 non-null      int64 
 5   환자 만 나이   2 non-null      int64 
 6   연도 + 반기   2 non-null      object
 7   상병기호      2 non-null      object
 8   구분        2 non-null      object
 9   요양기호      2 non-null      object
dtypes: int64(2), object(8)
memory usage: 176.0+ bytes


In [205]:
dfForML['환자성별'] = dfForML['환자성별'].astype('str')

<ipython-input-205-57682d209255>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfForML['환자성별'] = dfForML['환자성별'].astype('str')


In [206]:
#학습시 사용했던 LabelEncoder 객체 사용

le1 = LabelEncoder()
le1.classes_ = np.load('./classes_구분.npy')
dfForML['구분'] = le1.fit_transform(dfForML['구분'])

le2 = LabelEncoder()
le2.classes_ = np.load('./classes_반기.npy')
dfForML['연도 + 반기'] = le2.fit_transform(dfForML['연도 + 반기'])

le3 = LabelEncoder()
le3.classes_ = np.load('./classes_보험자구분.npy')
dfForML['보험자구분'] = le3.fit_transform(dfForML['보험자구분'])

le4 = LabelEncoder()
le4.classes_ = np.load('./classes_분류코드(기관).npy')
dfForML['분류코드(기관)'] = le4.fit_transform(dfForML['분류코드(기관)'])

le5 = LabelEncoder()
le5.classes_ = np.load('./classes_상병기호.npy')
dfForML['상병기호'] = le5.fit_transform(dfForML['상병기호'])

le6 = LabelEncoder()
le6.classes_ = np.load('./classes_요양기호.npy')
dfForML['요양기호'] = le6.fit_transform(dfForML['요양기호'])

le7 = LabelEncoder()
le7.classes_ = np.load('./classes_진료과목.npy')
dfForML['진료과목'] = le7.fit_transform(dfForML['진료과목'])

le8 = LabelEncoder()
le8.classes_ = np.load('./classes_항코드(중분류).npy')
dfForML['항코드(중분류)'] = le8.fit_transform(dfForML['항코드(중분류)'])

le9 = LabelEncoder()
le9.classes_ = np.load('./classes_환자성별.npy')
dfForML['환자성별'] = le9.fit_transform(dfForML['환자성별'])

np.load = np_load_old

<ipython-input-206-f2b003241ddd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfForML['구분'] = le1.fit_transform(dfForML['구분'])
<ipython-input-206-f2b003241ddd>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfForML['연도 + 반기'] = le2.fit_transform(dfForML['연도 + 반기'])
<ipython-input-206-f2b003241ddd>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [207]:
X = dfForML

In [208]:
model = pickle.load(open('./lgbm_original age_based on best f1score.sav', 'rb'))

In [209]:
# best Threshold는 0.29692155636794143였으므로
classification_results = model.predict_proba(X)[:, 1] >= 0.29692155636794143

In [210]:
classification_results = classification_results.tolist()
classification_results

[False, False]

In [211]:
prob = model.predict_proba(X)

In [212]:
prob

array([[0.99770847, 0.00229153],
       [0.99883255, 0.00116745]])

In [213]:
# 사용자에게 보여줄 문구 출력
i = 0
for j in entryForML:
    k = classification_results[i]
    if k == False:
        print(str(j) + '번째 항목은 환불불가 항목으로 분류되었습니다. 그래도 신청하시겠습니까?')
    else:
        tmp = prob[i][1] * 100
        print(str(j) + '번째 항목은 환불가능성이 ' + str(round(tmp, 3)) + '%로 환불가능 항목으로 분류되었습니다. 신청을 원하시면 제출버튼을 눌러주세요.')
    i += 1

1번째 항목은 환불불가 항목으로 분류되었습니다. 그래도 신청하시겠습니까?
3번째 항목은 환불불가 항목으로 분류되었습니다. 그래도 신청하시겠습니까?
